<a href="https://colab.research.google.com/github/heraclex12/VN-Punc-Pretrained-LMs/blob/main/VNPunc_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import re
from tensorflow.keras.preprocessing.sequence import pad_sequences
os.chdir('drive/My Drive/vncorenlp/journal_hero')

In [ ]:
from google.colab import auth
auth.authenticate_user()

### Data

In [35]:
max_paragraph_leng = 128
max_char_leng = 20
batch_size = 64
WORD_EMBEDDING_SIZE = 512
VOCAB_WORD_SIZE = 60000
special_tokens = ['<NUM>', '<URL>', '<EMAIL>']
punctuation_marks = ['PERIOD', 'COMMA', 'COLON', 'QMARK', 'EXCLAM', 'SEMICOLON', 'O']
pad_token = '<PAD>'
pad_id = len(punctuation_marks)
eos_marks = ['PERIOD', 'QMARK', 'EXCLAM']

In [47]:
train_df = pd.read_csv('punctuation/Novels/train.txt', encoding='utf-8', sep=' ', names=['token', 'label'], keep_default_na=False)
test_df = pd.read_csv('punctuation/Novels/test.txt', encoding='utf-8', sep=' ', names=['token', 'label'], keep_default_na=False)
valid_df = pd.read_csv('punctuation/Novels/valid.txt', encoding='utf-8', sep=' ', names=['token', 'label'], keep_default_na=False)

In [48]:
def get_paragraphs(df):
  idx = 0
  n_tokens = len(df)
  paragraphs = []
  token_labels = []
  while idx < n_tokens and idx >= 0:
    sub_df = df.iloc[idx: min(idx+128, n_tokens)]
    end_idx = sub_df[sub_df.label.isin(eos_marks)].tail(1).index
    if end_idx.empty:
      end_idx = -1
      paragraph_df = df.iloc[idx:]
    else:
      end_idx = end_idx.item() + 1
      paragraph_df = df.iloc[idx: end_idx]

    numeric_labels = paragraph_df.label.apply(lambda l: punctuation_marks.index(l))
    paragraphs.append(paragraph_df.token.values.tolist())
    token_labels.append(numeric_labels.values.tolist())
    idx = end_idx
  return paragraphs, token_labels

train_pags, train_labels = get_paragraphs(train_df)
valid_pags, valid_labels = get_paragraphs(valid_df)
test_pags, test_labels = get_paragraphs(test_df)

In [49]:
# setting TPU
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
except ValueError:
  tpu = None
  print('Not use TPU')


if tpu:
  tf.config.experimental_connect_to_cluster(tpu)
  tf.tpu.experimental.initialize_tpu_system(tpu)
  strategy = tf.distribute.experimental.TPUStrategy(tpu)
else: # default strategy that works on CPU and single GPU
  strategy = tf.distribute.get_strategy()

INFO:tensorflow:Initializing the TPU system: grpc://10.123.183.74:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.123.183.74:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [50]:
import collections
import unicodedata
import string

__SPIECE_UNDERLINE__ = u"▁"

def load_vocab(filename):
    """
      Load saved vocab file.
    """
    vocab = collections.OrderedDict()
    index = 0
    with tf.io.gfile.GFile(filename, "r") as reader:
        while True:
            token = reader.readline().strip()
            if not token:
                break
            token = token.split("\t")[0]
            vocab[token] = index
            index += 1
    return vocab

class Tokenizer():
    """
      Load Word vocabulary and Character vocabulary and convert ids to tokens end otherwise
    """
    def __init__(self, vocab=None, char_vocab=None, do_lower_case=False, unk_id = 1):
        
        if vocab:
            self.vocab = load_vocab(vocab)
            self.inv_vocab = {v: k for k, v in self.vocab.items()}

        if char_vocab:
            self.char_vocab = load_vocab(char_vocab)
            self.inv_char_vocab = {v: k for k, v in self.char_vocab.items()}

        self.max_input_chars_per_word = 200
        self.unk_id = unk_id
        self.do_lower_case = do_lower_case

    def tokenize(self, text, return_tok_to_orig=True):
        text = unicodedata.normalize("NFKC", text)
        if self.do_lower_case:
            text = text.lower()
        result = dict()
        tok_to_orig = []
        tokens = []
        token = ""
        def _tokenize_token(token):
            if token.isnumeric():
              token = '<NUM>'
            tokens.append(token)
            tok_to_orig.append([i-len(token), i])
        for i, c in enumerate(text):
            if _is_whitespace(c):
                if token:
                     _tokenize_token(token); token = ""
            elif _is_punctuation(c):
                if token:
                     _tokenize_token(token); token = ""
                tokens.append(c)
                tok_to_orig.append([i, i+1])
            else:
                token += c
        if token:
            i = len(text)
            _tokenize_token(token); token = ""
        if not return_tok_to_orig:
            return tokens
        return {
            "tokens": tokens,
            "tok_to_orig": tok_to_orig
        }

    def convert_tokens_to_ids(self, tokens, pad=False):
        token_ids = [self.vocab[token] if token in self.vocab else self.unk_id for token in tokens ]
        if pad:
          n_tokens = len(token_ids)
          max_len = min(max_paragraph_leng, n_tokens)
          return token_ids[:max_len] + [0] * (max_paragraph_leng - max_len)
          
        return token_ids

    def convert_ids_to_tokens(self, ids):
        return [self.inv_vocab[idd] for idd in ids]

    def convert_chars_to_ids(self, tokens, pad=False):
        def pad_char(char_id):
          n_char = len(char_id)
          max_len = min(max_char_length, n_char)
          return char_id[:max_len] + [0] * (max_char_length - max_len)

        if pad:
          char_ids = [[self.char_vocab[char] if char in self.char_vocab else self.unk_id for char in token] for token in tokens]
          char_ids = list(map(pad_char, char_ids))
          max_len = min(max_seq_length, len(tokens))
          char_ids = char_ids + [[0] * max_char_length for _ in range(max_seq_length - max_len)]
        else:
          char_ids = [[self.char_vocab[char] if char in self.char_vocab else self.unk_id for char in token] for token in tokens]

        return char_ids


    def convert_ids_to_chars(self, ids):
        return [[self.inv_char_vocab[idd] for idd in token] for token in ids]

def _is_punctuation(char):
    """Checks whether `chars` is a punctuation character."""
    cp = ord(char)
    if char in string.punctuation:
        return True
    cat = unicodedata.category(char)
    if cat.startswith("P") or cat == "Sc" or cat == "Sk":
        return True
    if ((cp >= 33 and cp <= 47) or (cp >= 58 and cp <= 64) or
        (cp >= 91 and cp <= 96) or (cp >= 123 and cp <= 126)):
        return True
    return False

def _is_whitespace(char):
    """Checks whether `chars` is a whitespace character."""
    if char == " " or char == "\t" or char == "\n" or char == "\r":
        return True
    cat = unicodedata.category(char)
    if cat == "Zs" or cat == "Cc" or cat == "Cf":
        return True
    return False


tokenizer = Tokenizer(vocab='gs://spellingchecker/vocab/vocab_punc_uncased.vocab', char_vocab='gs://spellingchecker/vocab/vcorrector-uncased-char.vocab')

In [51]:
train_input_ids = [tokenizer.convert_tokens_to_ids(train_pag, pad=True) for train_pag in train_pags]
test_input_ids = [tokenizer.convert_tokens_to_ids(test_pag, pad=True) for test_pag in test_pags]
valid_input_ids = [tokenizer.convert_tokens_to_ids(valid_pag, pad=True) for valid_pag in valid_pags]
train_input_ids = np.array(train_input_ids)
valid_input_ids = np.array(valid_input_ids)
test_input_ids = np.array(test_input_ids)

train_pad_labels = pad_sequences(train_labels, maxlen=max_paragraph_leng, padding='post', truncating='post', value=pad_id)
valid_pad_labels = pad_sequences(valid_labels, maxlen=max_paragraph_leng, padding='post', truncating='post', value=pad_id)
test_pad_labels = pad_sequences(test_labels, maxlen=max_paragraph_leng, padding='post', truncating='post', value=pad_id)

### Word-level 2xBi-LSTM Model

In [52]:
def categorical_crossentropy_loss(y_actual, y_pred):
  """
    Custom Categorical Crossentropy Loss function.
  """
  one_hot_correct = tf.one_hot(y_actual, depth=len(punctuation_marks), dtype=tf.float32)
  log_pred = tf.keras.backend.log(y_pred)
  per_example_loss = - tf.reduce_sum(log_pred * one_hot_correct, axis=[-1])
  positions_mask = tf.cast(tf.not_equal(y_actual, pad_id), tf.float32)
  loss_correct = tf.reduce_sum(per_example_loss * positions_mask, axis=[-1]) / (tf.reduce_sum(positions_mask, axis=[-1]) + 1e-5)
  return loss_correct
  

In [53]:
# Punctuation prediction model

# load model in TPU scope
with strategy.scope():
  # Initialize model
  model = tf.keras.Sequential()
  model.add(tf.keras.layers.Input(shape=(max_paragraph_leng,)))
  model.add(tf.keras.layers.Embedding(VOCAB_WORD_SIZE, WORD_EMBEDDING_SIZE))
  model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True)))
  model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True)))
  model.add(tf.keras.layers.Dense(len(punctuation_marks), activation='softmax'))

  model.compile(optimizer='adam', loss=categorical_crossentropy_loss)           # build model. Note: Must compile model in TPU scope. However, must call fit function out of scope


# Train
model.summary()
model.fit(train_input_ids, train_pad_labels, batch_size=batch_size, epochs=10, validation_data=(valid_input_ids, valid_pad_labels))

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 128, 512)          30720000  
_________________________________________________________________
bidirectional_6 (Bidirection (None, 128, 256)          656384    
_________________________________________________________________
bidirectional_7 (Bidirection (None, 128, 256)          394240    
_________________________________________________________________
dense_3 (Dense)              (None, 128, 7)            1799      
Total params: 31,772,423
Trainable params: 31,772,423
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
  1/179 [..............................] - ETA: 7:20 - loss: 1.9465WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0027s vs `on_train_batch_end` time: 0.0392s). Check you

179/179 [==============================] - ETA: 0s - loss: 0.5103WARNING:tensorflow:Callbacks method `on_test_batch_end` is slow compared to the batch time (batch time: 0.0015s vs `on_test_batch_end` time: 0.0144s). Check your callbacks.


179/179 [==============================] - 15s 84ms/step - loss: 0.5103 - val_loss: 0.3565
Epoch 2/10
179/179 [==============================] - 8s 45ms/step - loss: 0.3309 - val_loss: 0.3093
Epoch 3/10
179/179 [==============================] - 8s 45ms/step - loss: 0.2947 - val_loss: 0.2913
Epoch 4/10
179/179 [==============================] - 8s 46ms/step - loss: 0.2704 - val_loss: 0.2835
Epoch 5/10
179/179 [==============================] - 8s 46ms/step - loss: 0.2487 - val_loss: 0.2686
Epoch 6/10
179/179 [==============================] - 8s 46ms/step - loss: 0.2288 - val_loss: 0.2654
Epoch 7/10
179/179 [==============================] - 8s 46ms/step - loss: 0.2105 - val_loss: 0.2577
Epoch 8/10
179/179 [==============================] - 9s 49ms/step - loss: 0.1938 - val_loss: 0.2600
Epoch 9/10
179/179 [==============================] - 8s 46ms/step - loss: 0.1777 - val_loss: 0.2642
Epoch 10/10
179/179 [==============================] - 8s 46ms/step - loss: 0.1624 - val_loss: 0.2675

### Evanluation

In [54]:
from sklearn.metrics import f1_score, precision_score, recall_score, classification_report
def evaluate(y_actual, y_pred, average=None, is_logits=True):
  """
    Compute accuracy for n-classes, exclude padding tokens
  """
  if is_logits:
    y_pred = tf.reshape(tf.cast(tf.argmax(y_pred, axis=-1), tf.float32), (-1, y_actual.shape[-1]))
    y_pred = y_pred.numpy()
  y_actual = y_actual.tolist()
  pred_labels = y_pred.tolist()

  seq_lens = []
  not_pad_actual = []
  for y_seq in y_actual:
    cnt_len = 0
    for tag in y_seq:
      if tag != pad_id:
        cnt_len += 1
        not_pad_actual.append(tag)
      else:
        break
    seq_lens.append(cnt_len)

  pred_labels = [punctuation_marks[int(label)] for idx, labels in enumerate(pred_labels) for label in labels[:seq_lens[idx]]]
  not_pad_actual = [punctuation_marks[label] for label in not_pad_actual]
  report = classification_report(not_pad_actual, pred_labels, labels=['PERIOD', 'COMMA', 'COLON', 'QMARK', 'EXCLAM', 'SEMICOLON'], digits=4)

  return report

In [55]:
pred_test = model.predict(test_input_ids)
report = evaluate(test_pad_labels, pred_test, average=None, is_logits=True)

print(report)

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

      PERIOD     0.5505    0.4536    0.4974     29643
       COMMA     0.4540    0.3352    0.3856     21231
       COLON     0.0000    0.0000    0.0000      1153
       QMARK     0.7118    0.5158    0.5982      5271
      EXCLAM     0.4798    0.4596    0.4695      9167
   SEMICOLON     0.0000    0.0000    0.0000        43

   micro avg     0.5217    0.4134    0.4613     66508
   macro avg     0.3660    0.2940    0.3251     66508
weighted avg     0.5128    0.4134    0.4569     66508

